<a href="https://colab.research.google.com/github/mehtamohit013/comms_lidar_ML/blob/main/RL/alt_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Only
Run once and restart runtime

In [1]:
!nvidia-smi
from google.colab import drive
drive.mount('/gdrive')

Wed Jan 12 11:53:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture _
!unzip /gdrive/MyDrive/BTP/webots_data/final/alt_data_split.zip
# !unzip /gdrive/MyDrive/BTP/Webots_data/val_data.zip
!pip install pytorch-lightning

In [3]:
!pip install  --upgrade pandas

     |████████████████████████████████| 11.3 MB 5.4 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import scipy.io as io
import tqdm
import math
import random
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import copy
import getpass


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard


import gym

HOME = os.environ['HOME']

In [2]:
user = getpass.getuser()

if user == 'root':  #For google colab
    dpath = '.'
    save_dir = './model_state_dict'
else:
    dpath = os.environ['HOME'] + '/webots_code/data/final'
    save_dir = os.path.join(os.environ['HOME'],'webots_code/model_state_dict')

# Gym Env

In [3]:
class BS_env(gym.Env):
    def __init__(self,ep:list,ds:str='train'):
        
        '''
        ep: Pass the number of episodes to run: [start,end] where end is inclusive 
        ds: Dataset type: 'train','val' and 'test'
        '''
        super().__init__()
        self.len = ep[1]
        self.curr = ep[0]
        self.reward_range = [-120,0]
        
        if ds == 'train':
            self.df = pd.read_pickle(dpath+'/train.pkl')
        elif ds == 'val':
            self.df = pd.read_pickle(dpath+'/val.pkl')
        else:
            self.df = pd.read_pickle(dpath+'/test.pkl')
        
        self.df = self.df.reset_index()
        
        #Observation Space: Lidar : [10,240,240]
        self.observation_space = gym.spaces.Box(
                                low = -10.0,
                                high = 10.0,
                                shape = (10,240,240),
                                dtype = np.float32
                                )
        
        # Action Space : Discrete : 0,1,2 -> Base station
        self.action_space = gym.spaces.Discrete(3)
        
        # Init the env and resetting it
        self.reset()
    
    def reset(self):
        self._state = np.zeros((10,240,240))
        self.curr = 0
    
    def step(self, action):
        
        '''
        Inp : Action Space
        Output : Next State , Reward, done, info
        '''
        label = io.loadmat(dpath+'/labels/'+self.df.at[self.curr,'Lidar'][:-3]+'mat')
        label = (np.array([label['ss'][0][0],
                                   label['ss'][1][0],
                                   label['ss'][2][0]]))
        self.state = np.load(dpath+'/lidar_compressed/'+self.df.at[self.curr,'Lidar'])['lidar']

        '''
        Reward Calculation
        Currently using power as reward
        '''
        reward = label[action]
#         print(type(reward))
        
        if reward <= -120.0 :
            reward = np.float64(-120.0)
        
        self.curr +=1
        
        if self.curr>=self.len:
            done=1
        else:
            done = 0
            
        info = [action,label]
        return self.state, reward, done, info   

In [4]:
# gym_env = BS_env([0,10],'train')
# gym_env.step(0)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
max_train = len(pd.read_pickle(dpath+'/train.pkl'))
max_val = len(pd.read_pickle(dpath+'/val.pkl'))
epi = [0,500] #[start,end] 
epi_val = [0,max_val-1]

gym_env = BS_env(epi,'train')
gym_env_val = BS_env(epi_val,'val')

n_steps = epi[1] - epi[0] + 1
n_steps_val = epi_val[1] - epi_val[0] + 1

Relevant data pts

    1. Car Model
    2. GPS coord 
    3. Speed

Combined with previous history vector can generate new history

In [7]:
pd.read_pickle('./train.pkl')

,Time,Name,Model,GPS,Speed,Lidar,BS
0,605.632,SUMO vehicle 47,Toyota Prius,"[[38.894820842098504, -77.07385995665486, 0.71...",3.113155,SUMO vehicle 47605.6.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
1,607.68,SUMO vehicle 47,Toyota Prius,"[[38.894884505269076, -77.07383196516334, 0.71...",4.794069,SUMO vehicle 47607.7.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
2,609.728,SUMO vehicle 47,Toyota Prius,"[[38.89499438634594, -77.07385109049616, 0.717...",8.103935,SUMO vehicle 47609.7.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
3,719.296,SUMO vehicle 47,Toyota Prius,"[[38.895045024934106, -77.07321083734907, 0.71...",8.571484,SUMO vehicle 47719.3.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
4,721.344,SUMO vehicle 47,Toyota Prius,"[[38.894916914843826, -77.0731866334507, 0.717...",6.881279,SUMO vehicle 47721.3.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
...,...,...,...,...,...,...,...
11995,6359.136,SUMO vehicle 21,Citroen C-Zero,"[[38.895026305107784, -77.07311495342095, 0.33...",6.006405,SUMO vehicle 216359.1.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
11996,6361.056,SUMO vehicle 21,Citroen C-Zero,"[[38.89514876008529, -77.07313505098564, 0.334...",7.776782,SUMO vehicle 216361.1.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
11997,6362.976,SUMO vehicle 21,Citroen C-Zero,"[[38.89531822234122, -77.07316212286345, 0.334...",10.350013,SUMO vehicle 216363.0.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."
11998,6387.68,SUMO vehicle 21,Citroen C-Zero,"[[38.89511082631392, -77.07322169759945, 0.334...",5.625618,SUMO vehicle 216387.7.npz,"[[[38.895, -77.07303, 5.0], [38.89442, -77.072..."


# Replay Memory

In [8]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# DQN

In [9]:
class DQN(nn.Module):

    def __init__(self,drp:float=0.3,
                 drp_fc:float=0.2):
        super().__init__()
        
        self.drop_CNN = nn.Dropout(drp)
        self.drop_fc = nn.Dropout(drp_fc)
        self.mpool = nn.MaxPool2d((2,2))
        self.channels = 5
        self.fchannel = 3
        self.conv1 = self.create_conv(10,self.channels,13)
        self.conv2 = self.create_conv(self.channels,self.channels,13)
        self.conv3 = self.create_conv(self.channels,self.channels,7)
        self.conv4 = self.create_conv(self.channels,self.channels,7)

        # 1st Inception block
        self.in1 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in2 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in3 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.channels,self.fchannel,1)
        )
        self.in4 = self.create_conv(self.channels,self.fchannel,1)

        # Second inception block
        self.in5 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in6 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in7 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.fchannel*4,self.fchannel,1)
        )
        self.in8 = self.create_conv(self.fchannel*4,self.fchannel,1)

        self.conv5 = self.create_conv(self.fchannel*4,self.fchannel,7)
        self.conv6 = self.create_conv(self.fchannel,self.fchannel,7)

        self.conv7 = self.create_conv(self.fchannel,self.fchannel,3)
        self.conv8 = self.create_conv(self.fchannel,self.fchannel,3)
        
        self.flatten = nn.Flatten()

        self.linear = nn.Sequential(
            nn.Linear(588,16),
            nn.PReLU(1),
#             nn.BatchNorm1d(16),
            nn.Linear(16,3)
        )

    
    def forward(self,X):
        X = X.to(device)
        X  = self.conv1(X)
        X  = self.conv2(X)
        X = self.mpool(X)

        X = self.conv3(X)
        X = self.conv4(X)
        X = self.mpool(X)

        X = self.drop_CNN(X)

        a = self.in1(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in2(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in3(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in4(X)

        X = torch.cat((a,b,c,d),1)

        X = self.drop_CNN(X)

        a = self.in5(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in6(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in7(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in8(X)

        X = torch.cat((a,b,c,d),1)
        X = self.drop_CNN(X)

        X = self.conv5(X)
        X = self.conv6(X)

        X = self.mpool(X)

        X = self.drop_CNN(X)

        X = self.conv7(X)
        X = self.conv8(X)
        # print(X.shape)

        X = self.flatten(X)
        out = self.linear(X)

        return out
    
    def create_conv(self,in_layers:int,out_layers:int,kernel,stride:int=1,
                    padding:int=0) -> nn.Module :

        return nn.Sequential(
            nn.Conv2d(in_layers,out_layers,kernel,stride,padding),
            nn.PReLU(out_layers),
            nn.BatchNorm2d(out_layers)
        )

In [10]:
# tmp = DQN()
# inp = torch.rand(1,10,240,240)
# print(inp.shape)
# tmp(inp)

# Hyper Params

In [11]:


BATCH_SIZE = 32
GAMMA = 0.99

EPS_START = 0.9
EPS_END = 0.2

# It depends on overall number of steps, basic intitution is that
# once steps_done == EPS_DECAY then the probablity of choosing 
# random action is 33%; considering EPS_END is zero
# As for ep = [0,10]; approx ep is 80k therefore exploration can be reduced to 33% around 50k
# Also because of this factor smoothed accuracy matters more for training then seeing the average
EPS_DECAY = n_steps*0.3

TARGET_UPDATE = 10
VAL_STEP = 100000
Replay = 15000

EPOCH = 10

# Choosing Action
Based on epsilon-greedy 

In [12]:
policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(500000)

n_actions = 3
steps_done = 0

def select_action(state,steps_done:int=0,val:bool=False):
    if val:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            flattened_action = policy_net(state).max(dim = 1).indices
            return torch.tensor([[flattened_action]], device=device, dtype=torch.long)
    else:
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
        if sample > eps_threshold:
            with torch.no_grad():
                # t.max(1) will return largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                flattened_action = policy_net(state).max(dim = 1).indices
                return torch.tensor([[flattened_action]], device=device, dtype=torch.long)
        else:
            flattened_action = random.randrange(n_actions)
            return torch.tensor([[flattened_action]], device=device, dtype=torch.long)

# Optimize Model

In [13]:
def optimize_model(memory:ReplayMemory):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()


# Training Loop

In [14]:
#  Tensorboard
!rm -rf ./DQN
log_dir = './DQN' 

writer = SummaryWriter(log_dir=log_dir)

In [ ]:
%tensorboard --logdir ./DQN

## Validation

In [16]:
def val(train_step:int,gym_env_val:gym.Env,n_steps_val:int=n_steps_val,
        writer:SummaryWriter=writer):

    state = torch.zeros((1,10,240,240), dtype=torch.float32)
    ovr_reward = 0
    pos = 0
    ovr = 0
    for i_episode in tqdm.notebook.tqdm(range(n_steps_val),desc='Validation'):
        # Select and perform an action
        action = select_action(state,val=True)
        # Observe new state
        next_state, reward, done, info = gym_env_val.step(action)

        ovr_reward+=reward.item()

        next_state = next_state.astype(np.float32)[None,:,:,:]
        next_state = torch.tensor(next_state)
        reward = torch.tensor([reward], device=device).float()

        # Move to the next state
        state = next_state

        ovr+=1
        if np.argsort(info[1])[info[0]] == 0:
            pos+=1
        
        if done:
            break
        
    writer.add_scalar('Val Overall Reward',ovr_reward,train_step)
    writer.add_scalar('Val Average Rewad',(ovr_reward/n_steps_val),train_step)

    print(f'Validation Overall reward = {ovr_reward:.2f}. ' \
        f' Validation Average Reward = {ovr_reward/n_steps_val:.4f}'\
        f' Validation Accuracy = {float(pos)/float(ovr)}')
    # gym_env_val.close()

## Training

In [17]:
# Initialize the environment and state
cnt=0
for i in range(0,EPOCH):
    steps_done =0
    memory = ReplayMemory(Replay)

    gym_env.reset()
    state = torch.zeros((1,10,240,240), dtype=torch.float32)
    ovr_reward = 0

    ovr = 0
    pos = 0

    for i_episode in tqdm.notebook.tqdm(range(n_steps),desc='Train'):
        # Select and perform an action
        action = select_action(state,steps_done)
        # Observe new state
        next_state, reward, done, info = gym_env.step(action)
        if reward.item() == -np.inf :
            print(info,i_episode)
            break
        ovr_reward+=reward.item()

        next_state = next_state.astype(np.float32)[None,:,:,:]
        next_state = torch.tensor(next_state)
        reward = torch.tensor([reward], device=device).float()

        writer.add_scalar('episode_reward',reward,n_steps*cnt+steps_done)

        # print(next_state)
        # print(state)
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model(memory)
        
        # Update the target network, copying all weights and biases in DQN
        if i_episode % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if (i_episode) == 0:
            gym_env_val.reset()
            val(cnt, gym_env_val)
        steps_done+=1

        ovr+=1
        if np.argsort(info[1])[info[0]] == 0:
            pos+=1
        
        if done:
            break
        
    cnt+=1
    print(f'Overall Train reward after epoch {i} = {ovr_reward:.2f}. ' \
        f'Average Reward = {ovr_reward/n_steps:.4f} '\
        f'Training Accuracy = {float(pos)/float(ovr)}')
#     gym_env.close()

writer.add_hparams(
        {'BATCH_SIZE' : BATCH_SIZE,
        'GAMMA' : GAMMA,
        'EPS_START' : EPS_START,
        'EPS_END' : EPS_END,
        'EPS_DECAY' : EPS_DECAY,
        'TARGET_UPDATE' :TARGET_UPDATE},
        {
            'Overall Reward':ovr_reward,
            'Average Reward': ovr_reward/n_steps
        }
    )

val(cnt)

Train:   0%|          | 0/501 [00:00<?, ?it/s]

Validation:   0%|          | 0/1000 [00:00<?, ?it/s]

Validation Overall reward = -76076.32.  Validation Average Reward = -76.0763 Validation Accuracy = 0.3053053053053053
Overall Train reward after epoch 0 = -37830.69. Average Reward = -75.5104 Training Accuracy = 0.34


Train:   0%|          | 0/501 [00:00<?, ?it/s]

Validation:   0%|          | 0/1000 [00:00<?, ?it/s]

Validation Overall reward = -75518.73.  Validation Average Reward = -75.5187 Validation Accuracy = 0.3063063063063063


KeyboardInterrupt: ignored